In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow.keras as keras

# Set inputShape and imageSize to appropriate values
inputShape = (512, 512)  # For example, using 128x128 square images
imageSize = 512
batchSize = 32 
base_directory = "C:/Users/yosse/Downloads/Brain Tumor Data Set/Brain Tumor Data Set"

dataGen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

x_y_train = dataGen.flow_from_directory(
    base_directory,
    target_size=inputShape,
    batch_size=batchSize,
    shuffle=True,
    color_mode="grayscale",
    class_mode="binary",
    subset="training"
)

x_y_test = dataGen.flow_from_directory(
    base_directory,
    target_size=inputShape,
    batch_size=batchSize,
    shuffle=True,
    color_mode="grayscale",
    class_mode="binary",
    subset="validation"
)

labels = x_y_train.class_indices
labels_list = list(labels.keys())

KerasModel = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(imageSize, imageSize, 1)),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
    keras.layers.MaxPool2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

KerasModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print('Model Details are:')
print(KerasModel.summary())

epochs = 10

ThisModel = KerasModel.fit(x_y_train, epochs=epochs, verbose=1, validation_data=x_y_test)

ModelLoss, ModelAccuracy = KerasModel.evaluate(x_y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy))

pred = KerasModel.predict(x_y_test)
pred = np.squeeze((pred > 0.5).astype(int))

y_test = x_y_test.classes

def plot_image(i, predictions, img_test):
    pred_i, true_label, img = predictions[i], img_test[1][1][i], img_test[1][0][i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap='gray')

    if pred_i == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(labels_list[pred_i],
                                          100 * np.max(pred_i),
                                          labels_list[int(true_label)]),
               color=color)

plt.figure(figsize=(20, 20))
for i in range(0, 25, 1):
    plt.subplot(5, 5, i + 1)
    plot_image(i, pred, x_y_test)
plt.show()

print(classification_report(y_test, pred))
print("Accuracy of the Model:", accuracy_score(y_test, pred) * 100, "%")

conf_matrix = confusion_matrix(y_true=y_test, y_pred=pred)
plt.figure(figsize=[4, 4])
plt.imshow(conf_matrix, cmap="bone_r")
plt.title("Confusion matrix", fontsize=16)
plt.ylabel("Actual classes")
plt.xlabel("Predicted classes")
tick_marks = np.arange(2)
plt.yticks(tick_marks, labels_list)
plt.xticks(tick_marks, labels_list)
plt.colorbar()
for i, j in itertools.product(range(2), range(2)):
    plt.text(j, i, conf_matrix[i][j], horizontalalignment="center",
             color="white" if conf_matrix[i][j] > 100 else "black")
plt.show()

# history: the training history of the model (e.g., output of model.fit())
train_loss = ThisModel.history['loss']
train_acc = ThisModel.history['accuracy']
val_loss = ThisModel.history['val_loss']
val_acc = ThisModel.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

# plot training and validation loss
plt.plot(epochs, train_loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.figure()

# plot training and validation accuracy
plt.plot(epochs, train_acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
